In [6]:
# -*- coding: utf-8 -*-
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 21 15:20:11 2018

@author: Kevin Lu
"""

import pandas as pd
import numpy as np
import time
import pulp  

processing_time=pd.read_excel("Jobshop.xlsx",
                              sheet_name="processing_time",
                              header = None)
machine_order=pd.read_excel("Jobshop.xlsx",
                            sheet_name="machine_order",
                            header = None)

pt = processing_time.values
mo = machine_order.values

model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)

dv = pulp.LpVariable.dicts("start_time",
                                     ((i, j) for i in range(6) for j in range(6)),
                                     lowBound=0,
                                     cat='Continuous')
b =  pulp.LpVariable.dicts("binary_var",
                                     ((i, j,o) for i in range(6) for j in range(6) for o in range(6) if j<o),
                                     lowBound=0,
                                     cat='Binary')

Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')


model += Cmax

for j in range(6):
    for i in range(5):
       I = mo[j,i]
       K = mo[j,i+1]
       J = j
       model += (dv[K,J] - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for j in range(6):
    for i in range(6):
       I = mo[j,i]
       J = j
       model += (Cmax - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for i in range(6):
    for j in range(6):
        for l in range(6):
            if j<l:
                I=i
                L = l
                J = j
                model += (dv[I,J] - dv[I,L]) >= (pt[L][list(mo[L]).index(I)] - 100*(1-b[i,J,L]))
                model += (dv[I,L] - dv[I,J]) >= (pt[J][list(mo[J]).index(I)] - 100*(b[i,J,L]))
       
start_time = time.time()
model.solve()
pulp.LpStatus[model.status]     

for var in dv:
    var_value = dv[var].varValue
    print( var[0],'-',var[1] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print('the elapsed time:%s'% (time.time() - start_time))

0 - 0 13.0
0 - 1 40.0
0 - 2 21.0
0 - 3 16.0
0 - 4 50.0
0 - 5 30.0
1 - 0 16.0
1 - 1 0.0
1 - 2 37.0
1 - 3 11.0
1 - 4 22.0
1 - 5 8.0
2 - 0 5.0
2 - 1 8.0
2 - 2 0.0
2 - 3 22.0
2 - 4 13.0
2 - 5 49.0
3 - 0 35.0
3 - 1 50.0
3 - 2 7.0
3 - 3 27.0
3 - 4 54.0
3 - 5 11.0
4 - 0 49.0
4 - 1 13.0
4 - 2 38.0
4 - 3 30.0
4 - 4 25.0
4 - 5 45.0
5 - 0 42.0
5 - 1 28.0
5 - 2 11.0
5 - 3 45.0
5 - 4 38.0
5 - 5 19.0
min cost: 55.0
the elapsed time:3.5792057514190674


In [7]:
'''--------plot gantt chart-------'''
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import datetime

m_keys=[j for j in range(6)]
j_keys=[j for j in range(6)]
key_count={key:0 for key in j_keys}
j_count={key:0 for key in j_keys}
m_count={key:0 for key in m_keys}
j_record={}
#for i in sequence_best:
#    gen_t=int(pt[i][key_count[i]])
#    gen_m=int(ms[i][key_count[i]])
#    j_count[i]=j_count[i]+gen_t
#    m_count[gen_m]=m_count[gen_m]+gen_t
#    
#    if m_count[gen_m]<j_count[i]:
#        m_count[gen_m]=j_count[i]
#    elif m_count[gen_m]>j_count[i]:
#        j_count[i]=m_count[gen_m]
for var in dv:
    var_value = dv[var].varValue
    
    start_time=str(datetime.timedelta(seconds=var_value)) # convert seconds to hours, minutes and seconds
    end_time=str(datetime.timedelta(seconds=var_value+pt[var[1]][list(mo[var[1]]).index(var[0])]))
        
    j_record[(var[1],var[0])]=[start_time,end_time]
    
    key_count[i]=key_count[i]+1
        

df=[]
for m in m_keys:
    for j in j_keys:
        df.append(dict(Task='Machine %s'%(m), Start='2018-11-26 %s'%(str(j_record[(j,m)][0])), Finish='2018-11-26 %s'%(str(j_record[(j,m)][1])),Resource='Job %s'%(j)))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Job shop Schedule')
py.iplot(fig, filename='job_shop_scheduling', world_readable=True)

In [5]:
j_record

{(0, 0): ['0:00:05', '0:00:08'],
 (1, 0): ['0:00:39', '0:00:49'],
 (2, 0): ['0:00:27', '0:00:36'],
 (3, 0): ['0:00:13', '0:00:18'],
 (4, 0): ['0:00:44', '0:00:47'],
 (5, 0): ['0:00:33', '0:00:43'],
 (0, 1): ['0:00:19', '0:00:25'],
 (1, 1): ['0:00:10', '0:00:18'],
 (2, 1): ['0:00:37', '0:00:38'],
 (3, 1): ['0:00:00', '0:00:05'],
 (4, 1): ['0:00:27', '0:00:30'],
 (5, 1): ['0:00:15', '0:00:18'],
 (0, 2): ['0:00:00', '0:00:01'],
 (1, 2): ['0:00:16', '0:00:21'],
 (2, 2): ['0:00:05', '0:00:10'],
 (3, 2): ['0:00:20', '0:00:25'],
 (4, 2): ['0:00:15', '0:00:24'],
 (5, 2): ['0:00:41', '0:00:42'],
 (0, 3): ['0:00:32', '0:00:39'],
 (1, 3): ['0:00:42', '0:00:46'],
 (2, 3): ['0:00:13', '0:00:17'],
 (3, 3): ['0:00:29', '0:00:32'],
 (4, 3): ['0:00:47', '0:00:48'],
 (5, 3): ['0:00:19', '0:00:22'],
 (0, 4): ['0:00:41', '0:00:47'],
 (1, 4): ['0:00:30', '0:00:40'],
 (2, 4): ['0:00:50', '0:00:57'],
 (3, 4): ['0:00:33', '0:00:41'],
 (4, 4): ['0:00:37', '0:00:42'],
 (5, 4): ['0:00:40', '0:00:44'],
 (0, 5): [